Importer les packages

In [106]:
# importer les modules python nécessaires
from itertools import product
import os
from collections import defaultdict
import ViennaRNA

# Pour lire le fichier fasta
from Bio import SeqIO

import subprocess

# importer numpy
import numpy as np

# importer les modules scikit-learn nécessaires
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score


1. Repliement avec RNAfold

In [175]:

def executer_rnafold(fichier_fasta, fichier_output):
    with open(fichier_output, "w") as output_file:
        for seq_record in SeqIO.parse(fichier_fasta, "fasta"):
            sequence = str(seq_record.seq)
            process = subprocess.Popen(['RNAfold'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            [stdout, stderr] = process.communicate(input=sequence)
            output_file.write(f">{seq_record.id}\n")  # Write the sequence name
            output_file.write(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")  # Write the RNAfold output
        
        # print(f"{stdout.split()[0]}\n{stdout.split()[1]}\n")

In [174]:
fasta_files = ["Fasta/hsa_hairpin.fasta", "Fasta/hsa_hairpin_neg1.fasta", "Fasta/hsa_hairpin_neg2.fasta"]

for file in fasta_files :
    file_output = f"Fasta/repliments/{file.split('/')[1].split('.')[0]}_fold.fasta"
    executer_rnafold(file, file_output)

UCUCCGUUUAUCCCACCACUGCCACCAUUAUUGCUACUGUUCAGCAGGUGCUGCUGGUGGUGAUGGUGAUAGUCUGGUGGGGGCGGUGG
.(.((((...(((((((((((.((((((((((((.......((((((...)))))))))))))))))).))))..))))))))))).).

UGCUGAAAAUAGCCGACCUCCCGGGUAAUCCGGACACGCUCCGAUAGCCACCACCCAGAUGUUACCG
.((((....((.(((......))).))...((((.....)))).))))...................

AUAUUGAACUCGAGUUGGAAGAGGCGAGUCCGGUCUCAAAGUGACGACUGGCCCCGCCUCUUCCUCUCGGUCCC
.......((.((((..((((((((((.(.(((((((((...))).)))))).).)))))))))).))))))...



In [120]:
def traiter_fasta_fold(fichier_fold):
    
    repliments = defaultdict(list)

    # 1. Lire le fichier fichier_fold et récupérer son contenu dans une liste lignes
    with open(fichier_fold, 'r') as file:
        lignes = file.readlines()

    # supprimer "\n" à la fin des lignes
    lignes_propres = [ligne.strip() for ligne in lignes]

    # 2. Traiter repliement_fasta pour créer un dictionnaire de listes defaultdict(list)
    i = 0
    while i < len(lignes_propres):
        if lignes_propres[i].startswith('>'):
            nom_sequence = lignes_propres[i][1:]
            # mettre en lowercase
            sequence = lignes_propres[i+1].lower()
            structure = lignes_propres[i+2]
            repliments[nom_sequence] = [sequence, structure]
            i += 3
        else:
            i += 1

    return repliments
   


In [136]:
output_path = "Fasta/repliments/"
#repliments = []
repliments = defaultdict()

for file in os.listdir(output_path):
    name = f"dict{file.split('hairpin')[1].split('_rep')[0]}"
    #repliments.append(traiter_fasta_fold(f"{output_path}{file}"))
    repliments[name] = traiter_fasta_fold(f"{output_path}{file}")
    
# print(repliments.keys())

dict_keys(['dict_neg1', 'dict_neg2', 'dict'])


2. Calcul du nombre d’occurrences des triplets

In [75]:

def get_all_triplets():    

    db = ["...", "..(", ".((", "(((", "((.", "(..", "(.(", ".(."]
    nd = ["a", "c", "g", "u"]

    return [i+j for i,j in product(nd, db)]

In [76]:
# Generer les seauence RNQ et leur structures
triplets = get_all_triplets()

In [80]:

def get_seq_triplets(sequence, structure):
   
    seq_triplets = defaultdict(int)

    assert len(sequence) == len(structure)

    struct = re.sub("\)", "(", structure)

    seq_tab = list(sequence)
    stc_tab = list(struct)

    for ind in range(len(seq_tab)):
        if ind == 0: 
            before = "."
        else: 
            before = stc_tab[ind-1]

        midlle = stc_tab[ind]

        if ind == len(seq_tab)-1: 
            after = "."
        else:
            after = stc_tab[ind+1]

        triplet = seq_tab[ind] + before + midlle + after
        seq_triplets[triplet] += 1

    return seq_triplets

In [144]:
# Calculer les occurences des triplets dans les sequences et structures
for repliment in repliments.values() :
    #print(repliment)
    for sequence, structure in repliment.values():
        seq_triplets = get_seq_triplets(sequence, structure)
        #print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")

In [85]:

def calculer_Xu_triplets(repliements):
   
    liste_triplets = get_all_triplets()

    nb_sequences = len(repliements)
    nb_triplets = len(liste_triplets)

    triplets = np.zeros((nb_sequences, nb_triplets))
    
    for i, nom in enumerate(repliements):
        seq_triplets = get_seq_triplets(repliements[nom][0], repliements[nom][1])
        
        for j, triplet in enumerate(liste_triplets):
            if triplet in seq_triplets:
                triplets[i, j] = seq_triplets[triplet]
    
    return triplets

In [159]:
# Cette cellule est dédiée pour tester calculer_Xu_triplets()
Xu_triplets = defaultdict()
for repliment, name in zip( repliments.values(), repliments.keys() ):
    #print(name)
    Xu_triplets[name] = calculer_Xu_triplets(repliment)
print(Xu_triplets.keys())

dict_keys(['dict_neg1', 'dict_neg2', 'dict'])


3. Construction du jeu de données d'entrainement

In [87]:

def construire_dataset(data1, data2):
        
    # 1. Construire X

    X = np.concatenate((data1, data2), axis=0)
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)

    # 2. Mettre à l'échelle les valeurs de X entre 0 et 1 avec la méthode min-max

    X = (X - X_min) / (X_max - X_min)

    # 3. Construire y
    
    y = np.concatenate((np.zeros(data1.shape[0]), np.ones(data2.shape[0])))

    assert X.shape[0] == y.shape[0]

    return X, y


In [158]:
# Est-ce on a besoin des deux autres fichiers d'hairpin ? 
# mais ils sont les deux de classe negative
# Data 1
#executer_rnafold("Fasta/hsa_hairpin_neg1.fasta", "Fasta/repliments/hsa_hairpin_neg1_repliments.txt")
#data1dict = traiter_fasta_fold("Fasta/repliments/hsa_hairpin_neg1_repliments.txt")
#for sequence, structure in data1dict.values():
    #seq_triplets = get_seq_triplets(sequence, structure)
    # print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")
#data1 = calculer_Xu_triplets(data1dict)


In [90]:
#executer_rnafold("Fasta/hsa_hairpin_neg2.fasta", "Fasta/repliments/hsa_hairpin_neg2_repliments.txt")
#data2dict = traiter_fasta_fold("Fasta/repliments/hsa_hairpin_neg2_repliments.txt")
#for sequence, structure in data1dict.values():
    #seq_triplets = get_seq_triplets(sequence, structure)
    # print(f"Occurrences of triplets in sequence {sequence}: {seq_triplets}")
#data2 = calculer_Xu_triplets(data1dict)

AUAUUGAACUCGAGUUGGAAGAGGCGAGUCCGGUCUCAAAGUGACGACUGGCCCCGCCUCUUCCUCUCGGUCCC
.......((.((((..((((((((((.(.(((((((((...))).)))))).).)))))))))).))))))...



In [162]:
x,y = construire_dataset(Xu_triplets["dict"], Xu_triplets["dict_neg1"])
#print(x)

4. Évaluation et entrainement des modèles supervisés

In [190]:
def evaluer_cv_entrainer(x, y, cv, modele="lsvc", mesure='weighted'):
    
    # Modèles acceptés :
    modeles = {
        "lsvc": SVC(kernel='linear'),
        "nsvc": SVC(kernel='rbf'),
        "mnb": MultinomialNB(),
        "gnb": GaussianNB(),
        "dtc": DecisionTreeClassifier()
    }
    
    if modele not in modeles:
        print("Type de modele {} est invalide".format(modele))
        modele = "lsvc"
        print("Le modele {} sera entrainé".format(modele))
    
    # 1. Initialiser une instance de classifieur selon la valeur de modele
    classifieur = modeles[modele]
    
    # 2. Calculer les scores de validation croisée du classifieur avec X et y
    scorer = make_scorer(f1_score, average=mesure)
    scores = cross_val_score(classifieur, x, y, cv=cv, scoring=scorer)
    
    # 3. Entrainer le classifieur avec X et y
    classifieur.fit(x, y)
    
    return scores, classifieur

In [166]:
# Évaluer et entraîner un classifieur avec validation croisée
scores, classifieur = evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='weighted')

5. Fonction principale

In [189]:

def entrainer_par_triplets(
    fasta_seq_pos,
    fasta_fold_pos,
    fasta_seq_neg,
    fasta_fold_neg,
    cv,
    modele="lsvc",
    mesure="f1_weighted"):
    

    ## 1. Replier les séquences du set positif
    # Replier les séquences si le fichier fasta_fold_pos n'existe pas
    if not os.path.isfile(fasta_fold_pos):
        executer_rnafold(fasta_seq_pos, fasta_fold_pos)
    
    ## 2. Replier les séquences du set négatif
    # Replier les séquences si le fichier fasta_fold_neg n'existe pas
    if not os.path.isfile(fasta_fold_neg):
        executer_rnafold(fasta_seq_neg, fasta_fold_neg)


    ## 3. Traiter les repliements du set positif 
    repliements_pos = traiter_fasta_fold(fasta_fold_pos)

    ## 4. Traiter les repliements du set négatif
    repliements_neg1 = traiter_fasta_fold(fasta_fold_neg)

    ## 5. Calculer les triplets du set positif
    triplets_pos = calculer_Xu_triplets(repliements_pos)

    ## 6. Calculer les triplets du set négatif
    triplets_neg1 = calculer_Xu_triplets(repliements_neg1)

    ## 7. Construire le jeu d'entrainement X et y
    x, y = construire_dataset(triplets_pos, triplets_neg1)

    ## 8. Évaluer et entrainer le classifieur avec validation croisée
    scores, classifieur = evaluer_cv_entrainer(x, y, cv=5, modele="lsvc", mesure='weighted')


    return scores, classifieur

In [169]:
# Cette cellule est dédiée pour tester entrainer_par_triplets()
fasta_seq_pos = "Fasta/hsa_hairpin.fasta"
fasta_fold_pos = "Fasta/repliments/hsa_hairpin_repliments.txt"
fasta_seq_neg = "Fasta/hsa_hairpin_neg1.fasta"
fasta_fold_neg = "Fasta/repliments/hsa_hairpin_neg1_repliments.txt"
scores, classifieur = entrainer_par_triplets(
                            fasta_seq_pos,
                            fasta_fold_pos,
                            fasta_seq_neg,
                            fasta_fold_neg,
                            cv=10,
                            modele="lsvc",
                            mesure="f1_weighted")

6. Comparaison des modèles

In [191]:
import os
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from statistics import mean, stdev


def comparer_modeles(
    fasta_seq_pos,
    fasta_fold_pos,
    fasta_seq_neg,
    fasta_fold_neg,
    cv,
    modeles=["lsvc", "nsvc", "mnb", "gnb", "dtc"],
    mesure="f1_weighted"):

    moyennes = []
    ecarts_types = []

    for modele in modeles:
        scores, _ = entrainer_par_triplets(fasta_seq_pos, fasta_fold_pos, fasta_seq_neg, fasta_fold_neg, cv, modele, mesure)
        moyennes.append(mean(scores))
        ecarts_types.append(stdev(scores))

    return moyennes, ecarts_types


In [172]:
moyennes, ecarts_types = comparer_modeles(
                fasta_seq_pos,
                fasta_fold_pos,
                fasta_seq_neg,
                fasta_fold_neg,
                cv=10,
                modeles=["lsvc", "nsvc", "mnb", "gnb", "dtc"],
                mesure="f1_weighted")

6.1 Comparer les modèles avec les précurseurs de l'humain

In [173]:
# Modèles à comparer :
comp_modeles = ["lsvc", "nsvc", "mnb", "gnb", "dtc"]

In [181]:
# Jeu de données 1
# ################
# séquences du set négatif générées par brassage aléatoire des nucléotides des séquences réelles

seq_pos = "Fasta/hsa_hairpin.fasta"
fold_pos = "Fasta/repliments/hsa_hairpin_fold.fasta"

seq_neg = "Fasta/hsa_hairpin_neg1.fasta"
fold_neg = "Fasta/repliments/hsa_hairpin_neg1_fold.fasta"

cv_iterations = 10
mesure_cv = "f1_weighted"

# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.861	0.016
nsvc	: 0.861	0.016
mnb	: 0.861	0.016
gnb	: 0.861	0.016
dtc	: 0.861	0.016


In [178]:
# Jeu de données 2
# ################
# séquences du set négatif générées par relocalisation de sous-séquences dans les séquences réelles

seq_pos = "Fasta/hsa_hairpin.fasta"
fold_pos = "Fasta/repliments/hsa_hairpin_fold.fasta"

seq_neg = "Fasta/hsa_hairpin_neg2.fasta"
fold_neg = "Fasta/repliments/hsa_hairpin_neg2_fold.fasta"

cv_iterations = 10
mesure_cv = "f1_weighted"

# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.639	0.010
nsvc	: 0.639	0.010
mnb	: 0.639	0.010
gnb	: 0.639	0.010
dtc	: 0.639	0.010


Question :
    Quelle méthode et quels modèles sont les plus efficaces pour discriminer entre les précurseurs réels et les pseudo-précurseurs ?

6.2 Comparer les modèles avec les précuseurs des animaux et des plantes

In [194]:
# Jeu de donnée 3
# ################

#seq_pos = "Fasta/Animal/bos_taurus_hairpin.fasta"
#fold_pos = "Fasta/Animal/repliments/bos_taurus_hairpin_fold.fasta"

seq_pos = "Fasta/Animal/mu_muscus_hairpin.fa"
fold_pos = "Fasta/Animal/repliments/mu_muscus_hairpin_fold.fa"

seq_neg = "Fasta/Plant/arabidopsis_thaliana_miRna.fasta"
fold_neg = "Fasta/Plant/repliments/arabidopsis_thaliana_miRna_fold.fa"

cv_iterations = 10
mesure_cv = "f1_weighted"


# Comparaison des modèles
moyennes, ecarts_types = comparer_modeles(seq_pos, fold_pos, seq_neg, fold_neg, cv_iterations)

print("{} de {}-fold validations croisées des modèles :\n".format(mesure_cv, cv_iterations))
print("modèle\t: mean\tstd")

for i, modele in enumerate(comp_modeles):
    print("{}\t: {:.3f}\t{:.3f}".format(modele, moyennes[i], ecarts_types[i]))

f1_weighted de 10-fold validations croisées des modèles :

modèle	: mean	std
lsvc	: 0.971	0.000
nsvc	: 0.971	0.000
mnb	: 0.971	0.000
gnb	: 0.971	0.000
dtc	: 0.971	0.000


Question
Les modèles entrainés avec les triplets peuvent-ils faire la discrimination entre les précurseurs des animaux et ceux des plantes ?